# Part 4: CNN Vision Training (50K Images)
Training a ResNet-inspired CNN on 50,000 candlestick chart images from 10 cryptocurrencies.

### CELL 1: Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf
print(f"🔥 GPU Available: {tf.config.list_physical_devices('GPU')}")

!pip install -q pandas numpy matplotlib tensorflow

### CELL 2: Load Chart Dataset

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

print("📥 Loading chart images (50,000)...")
# Load top 10 images
X = np.load('/content/drive/MyDrive/crypto_bot/data/chart_images_top10.npy')
y = np.load('/content/drive/MyDrive/crypto_bot/data/chart_labels_top10.npy')

print(f"✅ Loaded X: {X.shape}")
print(f"✅ Loaded y: {y.shape}")

# Normalize images
X = X.astype('float32') / 255.0

# One-hot encode labels
y_cat = to_categorical(y, num_classes=3)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_cat, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print(f"📊 Train size: {len(X_train):,}")
print(f"📊 Test size: {len(X_test):,}")

### CELL 3: Build & Train CNN

In [ ]:
from tensorflow.keras import layers, models, callbacks

def build_resnet_vision(input_shape=(224, 224, 3)):
    inputs = layers.Input(shape=input_shape)
    
    # Initial Conv Block
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(2)(x)
    
    # Residual Blocks
    for filters in [64, 128, 256]:
        shortcut = layers.Conv2D(filters, 1, padding='same')(x)
        
        res = layers.Conv2D(filters, 3, activation='relu', padding='same')(x)
        res = layers.BatchNormalization()(res)
        res = layers.Conv2D(filters, 3, padding='same')(res)
        res = layers.BatchNormalization()(res)
        
        x = layers.Add()([res, shortcut])
        x = layers.Activation('relu')(x)
        x = layers.MaxPooling2D(2)(x)
    
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(3, activation='softmax')(x)
    
    model = models.Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss='categorical_crossentropy', 
        metrics=['accuracy']
    )
    return model

model = build_resnet_vision()

checkpoint_path = '/content/drive/MyDrive/crypto_bot/models/cnn_top10_best.h5'
cb = [
    callbacks.EarlyStopping(patience=10, restore_best_weights=True),
    callbacks.ModelCheckpoint(checkpoint_path, save_best_only=True)
]

print("🚀 Training CNN on 50,000 images...")
history = model.fit(
    X_train, y_train, 
    validation_data=(X_test, y_test), 
    epochs=50, 
    batch_size=128, 
    callbacks=cb
)

### CELL 4: Save Final Model

In [ ]:
final_path = '/content/drive/MyDrive/crypto_bot/models/cnn_top10_final.h5'
model.save(final_path)
print(f"✅ Model saved to: {final_path}")